# Football dataset

## Importing the libraries

In [49]:
import sqlite3

import numpy as np
import pandas as pd

In [50]:
conn = sqlite3.connect('football.sqlite')
cursor = conn.cursor()

# Introduction

The primary objective of this project is to develop a predictive model that can outperform bookmakers' odds in predicting the outcome of a football match, specifically the home team's victory. This task is complex, involving the prediction of a binary outcome influenced by numerous factors. However, we are well-equipped to tackle this challenge with the comprehensive dataset from Kaggle, which includes data from 11 different leagues across 11 countries, 299 unique teams, 11,060 unique players, and 25,979 matches from 2008 to 2016. The ultimate goal is to develop a superior predictive model and commercialize our predictions, opening up a potentially lucrative business opportunity that could significantly outperform traditional betting strategies.

A secondary objective of this project is to construct a model that can accurately predict whether a football match will end with 'Under' or 'Over' 2.5 goals. This is a binary classification problem, and we aim to optimize the model for accuracy in predicting 'Under' outcomes.

To achieve these objectives, the project is carefully divided into several phases. The first phase, as outlined in the `data_cleaning` file, involves preparing the data for analysis. This includes importing necessary libraries, establishing a connection to the database, and extracting essential data from various tables. The data is then thoroughly cleaned by addressing null values, duplicates, and data types, ensuring the highest data quality for our analysis.

Following data cleaning, the project proceeds to the feature engineering phase. Here, we generate several new features expected to enhance the predictive power of our models. These features include the average rating of team players, the difference in home and away team ratings, the average player age, and the difference in home and away team ages. We also calculate the form of the home and away teams in the last five matches of the season, the team formations, and the rolling average of goals scored and conceded.

After completing feature engineering, we transition to the `eda_hypothesis` file, where we conduct exploratory data analysis and formulate hypotheses. This step is critical as it enables us to understand the data and identify potential features for our predictive models.

The core of our project is encapsulated in the `home_team_win_model` file, where we construct and evaluate various machine learning models. These models are trained on the cleaned and enriched data and evaluated using appropriate metrics. The model with the best performance is selected for further optimization. Additionally, we test our model's predictions against the bookmakers' odds for the 2015/2016 season. This allows us to evaluate the profitability of our model.

In the `goals_models` file, we concentrate on the specific predictive task of predicting whether a football match will end with 'Under' or 'Over' goals. We employ several machine learning algorithms to train our model. The model's performance is evaluated, and the best model is selected for further optimization. As we lack betting companies' odds for comparison, we evaluate our model using other metrics like ROC AUC score, ROC curve, and confusion matrix.

In conclusion, this project is a comprehensive and in-depth investigation into football match prediction. It presents a challenging yet rewarding endeavor, and we anticipate uncovering valuable insights and outperforming the odds. We delve into various aspects of the game, from team formations to player ratings, to construct a robust predictive model.

# Information about the dataset

## Countries and leagues

In [51]:
query = "SELECT * FROM league"
cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    print(row[2])

print(f"Count: {len(rows)}")

Belgium Jupiler League
England Premier League
France Ligue 1
Germany 1. Bundesliga
Italy Serie A
Netherlands Eredivisie
Poland Ekstraklasa
Portugal Liga ZON Sagres
Scotland Premier League
Spain LIGA BBVA
Switzerland Super League
Count: 11


In this database, there are 11 leagues from 11 different countries.

## Time period

In [52]:
query = "SELECT MIN(date), MAX(date) FROM match"
cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    print(f"Min date: {row[0]}, Max date: {row[1]}")

Min date: 2008-07-18 00:00:00, Max date: 2016-05-25 00:00:00


## Teams

- Unique teams

Let's check out how many unique teams played in these leagues during this time.

In [53]:
query = """
SELECT L.name AS league_name, COUNT(DISTINCT M.team_id) AS team_count
FROM (
    SELECT home_team_api_id AS team_id, country_id
    FROM Match
) M
JOIN League L ON M.country_id = L.country_id
GROUP BY L.name
"""

cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    print(f"League: {row[0]}, Team Count: {row[1]}")

query_total = "SELECT COUNT(DISTINCT home_team_api_id) FROM Match"
cursor.execute(query_total)

total_teams = cursor.fetchone()[0]

print(f"Total unique teams: {total_teams}")

League: Belgium Jupiler League, Team Count: 25
League: England Premier League, Team Count: 34
League: France Ligue 1, Team Count: 35
League: Germany 1. Bundesliga, Team Count: 30
League: Italy Serie A, Team Count: 32
League: Netherlands Eredivisie, Team Count: 25
League: Poland Ekstraklasa, Team Count: 24
League: Portugal Liga ZON Sagres, Team Count: 29
League: Scotland Premier League, Team Count: 17
League: Spain LIGA BBVA, Team Count: 33
League: Switzerland Super League, Team Count: 15
Total unique teams: 299


 - Team attributes

Let's check out the team attributes.

In [54]:
query = "SELECT * FROM Team_Attributes LIMIT 5"
team_attributes_5 = pd.read_sql_query(query, conn)
team_attributes_5

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


## Matches

- Count of matches

Let's find out how many games each league had. Knowing the number of games in each league is key because it gives us the sample size for our models.

In [55]:
query = """
SELECT L.name AS league_name, COUNT(*) AS match_count
FROM Match M
JOIN League L ON M.country_id = L.country_id
GROUP BY L.name
"""

cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    print(f"League: {row[0]}, Match Count: {row[1]}")

query_total = "SELECT COUNT(*) FROM Match"
cursor.execute(query_total)

total_matches = cursor.fetchone()[0]

print(f"Total matches: {total_matches}")

League: Belgium Jupiler League, Match Count: 1728
League: England Premier League, Match Count: 3040
League: France Ligue 1, Match Count: 3040
League: Germany 1. Bundesliga, Match Count: 2448
League: Italy Serie A, Match Count: 3017
League: Netherlands Eredivisie, Match Count: 2448
League: Poland Ekstraklasa, Match Count: 1920
League: Portugal Liga ZON Sagres, Match Count: 2052
League: Scotland Premier League, Match Count: 1824
League: Spain LIGA BBVA, Match Count: 3040
League: Switzerland Super League, Match Count: 1422
Total matches: 25979


-  Information about the matches

In [56]:
query = "SELECT * FROM Match LIMIT 5"
match_5 = pd.read_sql_query(query, conn)
match_5

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


## Players

- Unique players

In [57]:
query = "SELECT COUNT(DISTINCT player_api_id) FROM Player"
cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    print(f"Unique players: {row[0]}")

Unique players: 11060


In [58]:
query = "SELECT COUNT(*) FROM Player_Attributes"
cursor.execute(query)

rows = cursor.fetchone()

print(f"Total rows in Player_Attributes: {rows[0]}")

Total rows in Player_Attributes: 183978


- Player attributes

Let's check the players' attributes.

In [59]:
query = "SELECT * FROM Player_Attributes LIMIT 5"
player_attributes_5 = pd.read_sql_query(query, conn)
player_attributes_5

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
1,2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
2,3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,...,54,48,65,66,69,6,11,10,8,8
3,4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
4,5,218353,505942,2007-02-22 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7


### Summary of the dataset

This dataset contains information about football matches, teams, players, and player attributes. It has 11 leagues from 11 different countries. The time period is from 2008 to 2016. There are 299 unique teams and 11060 unique players. The total number of matches is 25979.



# Data Cleaning 

## Players 

First, we will merge the player and player attributes tables and will leave most important columns.

In [60]:
query = """
SELECT Player.player_api_id, Player.player_name, Player.birthday, Player.height, Player.weight, 
       Player_Attributes.overall_rating, Player_Attributes.potential, 
       Player_Attributes.attacking_work_rate, Player_Attributes.defensive_work_rate,
       Player_Attributes.date
FROM Player
JOIN Player_Attributes ON Player.player_api_id = Player_Attributes.player_api_id
"""
players = pd.read_sql_query(query, conn)

In [61]:
players.head()

,player_api_id,player_name,birthday,height,weight,overall_rating,potential,attacking_work_rate,defensive_work_rate,date
0,505942,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,67.0,71.0,medium,medium,2016-02-18 00:00:00
1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,67.0,71.0,medium,medium,2015-11-19 00:00:00
2,505942,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,62.0,66.0,medium,medium,2015-09-21 00:00:00
3,505942,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,61.0,65.0,medium,medium,2015-03-20 00:00:00
4,505942,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187,61.0,65.0,medium,medium,2007-02-22 00:00:00


- Null values

In [62]:
players.isnull().sum()

player_api_id             0
player_name               0
birthday                  0
height                    0
weight                    0
overall_rating          836
potential               836
attacking_work_rate    3230
defensive_work_rate     836
date                      0
dtype: int64

We will discard the rows that contain null values in the 'overall_rating' column. This column is vital for our analysis, and therefore, we do not need to retain rows with null values in this column.

In [63]:
players = players[players['overall_rating'].notnull()]

In [64]:
players.isnull().sum()

player_api_id             0
player_name               0
birthday                  0
height                    0
weight                    0
overall_rating            0
potential                 0
attacking_work_rate    2394
defensive_work_rate       0
date                      0
dtype: int64

- Duplicates

In [65]:
players.duplicated().sum()

0

- Date and birthday columns

We will convert the date and birthday columns to datetime data type. We will also calculate the age of the players at the time of the match.

In [66]:
players['birthday'] = pd.to_datetime(players['birthday'])
players['date'] = pd.to_datetime(players['date'])

players['age'] = players['date'] - players['birthday']

players['age'] = players['age'] / pd.Timedelta(days=365.25)

players['age'] = players['age'].astype(int)

In [67]:
players.head()

,player_api_id,player_name,birthday,height,weight,overall_rating,potential,attacking_work_rate,defensive_work_rate,date,age
0,505942,Aaron Appindangoye,1992-02-29,182.88,187,67.0,71.0,medium,medium,2016-02-18,23
1,505942,Aaron Appindangoye,1992-02-29,182.88,187,67.0,71.0,medium,medium,2015-11-19,23
2,505942,Aaron Appindangoye,1992-02-29,182.88,187,62.0,66.0,medium,medium,2015-09-21,23
3,505942,Aaron Appindangoye,1992-02-29,182.88,187,61.0,65.0,medium,medium,2015-03-20,23
4,505942,Aaron Appindangoye,1992-02-29,182.88,187,61.0,65.0,medium,medium,2007-02-22,14


## Matches

We will now extract important data from the Match,League,Team tables.

The data we are extracting includes:  
-Match details: match id, season, stage, date, home team goals, away team goals, and goal, and the home and away players ids.
-League details: Name of the league.
-Team details: Long names of the home and away teams.

In [68]:
query = """
SELECT Match.match_api_id, Match.season, Match.date, Match.home_team_goal, Match.away_team_goal,
       Match.stage,
       League.name AS league_name,
       home_team.team_long_name AS home_team,
       away_team.team_long_name AS away_team,
""" + ', '.join(f'Match.home_player_{i}, Match.away_player_{i}' for i in range(1, 12)) + ', ' + ', '.join(
    f'Match.home_player_y{i}, Match.away_player_y{i}' for i in range(1, 12)) + """
FROM Match
JOIN League ON Match.country_id = League.id
JOIN Team AS home_team ON Match.home_team_api_id = home_team.team_api_id
JOIN Team AS away_team ON Match.away_team_api_id = away_team.team_api_id
"""

matches = pd.read_sql_query(query, conn)

In [69]:
matches.head()

,match_api_id,season,date,home_team_goal,away_team_goal,stage,league_name,home_team,away_team,home_player_1,...,home_player_Y7,away_player_Y7,home_player_Y8,away_player_Y8,home_player_Y9,away_player_Y9,home_player_Y10,away_player_Y10,home_player_Y11,away_player_Y11
0,492473,2008/2009,2008-08-17 00:00:00,1,1,1,Belgium Jupiler League,KRC Genk,Beerschot AC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,492474,2008/2009,2008-08-16 00:00:00,0,0,1,Belgium Jupiler League,SV Zulte-Waregem,Sporting Lokeren,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,492475,2008/2009,2008-08-16 00:00:00,0,3,1,Belgium Jupiler League,KSV Cercle Brugge,RSC Anderlecht,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,492476,2008/2009,2008-08-17 00:00:00,5,0,1,Belgium Jupiler League,KAA Gent,RAEC Mons,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,492477,2008/2009,2008-08-16 00:00:00,1,3,1,Belgium Jupiler League,FCV Dender EH,Standard de Liège,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Null values

In [70]:
matches.isnull().sum()

match_api_id          0
season                0
date                  0
home_team_goal        0
away_team_goal        0
stage                 0
league_name           0
home_team             0
away_team             0
home_player_1      1224
away_player_1      1234
home_player_2      1315
away_player_2      1278
home_player_3      1281
away_player_3      1293
home_player_4      1323
away_player_4      1321
home_player_5      1316
away_player_5      1335
home_player_6      1325
away_player_6      1313
home_player_7      1227
away_player_7      1235
home_player_8      1309
away_player_8      1341
home_player_9      1273
away_player_9      1328
home_player_10     1436
away_player_10     1441
home_player_11     1555
away_player_11     1554
home_player_Y1     1821
away_player_Y1     1832
home_player_Y2     1821
away_player_Y2     1832
home_player_Y3     1832
away_player_Y3     1832
home_player_Y4     1832
away_player_Y4     1832
home_player_Y5     1832
away_player_Y5     1832
home_player_Y6  

There are a significant number of null values in the home and away player columns. We will analyze the null values by league to determine the extent of the missing data.

In [71]:
player_columns = [f'home_player_{i}' for i in range(1, 12)] + [f'away_player_{i}' for i in range(1, 12)]

null_counts = {
    'Null': matches[player_columns].isnull().groupby(matches['league_name']).sum().sum(axis=1),
    'Not Null': matches[player_columns].notnull().groupby(matches['league_name']).sum().sum(axis=1)
}

null_counts_df = pd.DataFrame(null_counts)
null_counts_df['Total'] = null_counts_df.sum(axis=1)
null_counts_df['Not Null Percentage'] = (null_counts_df['Not Null'] / null_counts_df['Total']) * 100
null_counts_df

,Null,Not Null,Total,Not Null Percentage
league_name,,,,
Belgium Jupiler League,5051,32965,38016,86.713489
England Premier League,80,66800,66880,99.880383
France Ligue 1,197,66683,66880,99.705443
Germany 1. Bundesliga,90,53766,53856,99.832888
Italy Serie A,279,66095,66374,99.579655
Netherlands Eredivisie,4776,49080,53856,91.131907
Poland Ekstraklasa,12286,29954,42240,70.913826
Portugal Liga ZON Sagres,3299,41845,45144,92.692274
Scotland Premier League,334,39794,40128,99.167663


We can see from the data that the leagues from Belgium, Netherlands, Poland, and Portugal have a significant amount of missing data regarding the players who participated in the matches.

Our data shows that the football leagues of Spain, England, Germany, France, Italy, and Scotland have the most comprehensive data, making them strong candidates for predictive modeling. However, we must also consider the players' skill levels across these leagues. We aim to avoid modeling significantly different players' skill levels within the same model. To this end, we will look to league levels later in the project. Then, we will decide which leagues can be grouped in the same model and which must be separated. This approach ensures a more accurate, suitable, and efficient analysis and better resource utilization compared to analyzing all leagues simultaneously.

- Duplicates

In [72]:
matches.duplicated().sum()

0

- Date column

In [73]:
matches['date'] = pd.to_datetime(matches['date'])

- Replacing player ID with overall rating 

In [74]:
players_sorted = players.sort_values('date', ascending=False)

player_ratings = players_sorted.set_index('player_api_id')['overall_rating'].to_dict()
player_ages = players_sorted.set_index('player_api_id')['age'].to_dict()

for column in player_columns:
    matches[column + '_age'] = matches[column].map(player_ages)
    matches[column] = matches[column].map(player_ratings)

- Result column

In [75]:
conditions = [
    (matches['home_team_goal'] > matches['away_team_goal']),
    (matches['home_team_goal'] < matches['away_team_goal']),
    (matches['home_team_goal'] == matches['away_team_goal'])
]

choices = ['Win', 'Lose', 'Draw']

matches['result'] = np.select(conditions, choices)

## Teams

We will now retrieve essential data from the Team and Team_attributes tables. We will limit our selection to only those teams that are part of the four leagues: Spain LIGA BBVA, England Premier League, Germany 1. Bundesliga, and Italy Serie A.

In [76]:
query = """
SELECT Team.team_long_name, Team_attributes.team_api_id, Team_attributes.date,
       Team_attributes.buildUpPlaySpeed, Team_attributes.buildUpPlaySpeedClass, Team_attributes.buildUpPlayDribbling,
       Team_attributes.buildUpPlayDribblingClass, Team_attributes.buildUpPlayPassing, Team_attributes.buildUpPlayPassingClass,
       Team_attributes.buildUpPlayPositioningClass, Team_attributes.chanceCreationPassing, Team_attributes.chanceCreationPassingClass,
       Team_attributes.chanceCreationCrossing, Team_attributes.chanceCreationCrossingClass, Team_attributes.chanceCreationShooting,
       Team_attributes.chanceCreationShootingClass, Team_attributes.chanceCreationPositioningClass, Team_attributes.defencePressure,
       Team_attributes.defencePressureClass, Team_attributes.defenceAggression, Team_attributes.defenceAggressionClass,
       Team_attributes.defenceTeamWidth, Team_attributes.defenceTeamWidthClass, Team_attributes.defenceDefenderLineClass
FROM Team_attributes
JOIN Team ON Team_attributes.team_api_id = Team.team_api_id
WHERE Team_attributes.team_api_id IN (
    SELECT home_team_api_id FROM Match WHERE country_id IN (1729, 7809, 10257, 21518)
    UNION
    SELECT away_team_api_id FROM Match WHERE country_id IN (1729, 7809, 10257, 21518)
)
"""
team_attributes = pd.read_sql_query(query, conn)

In [77]:
team_attributes.head()

,team_long_name,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,Milan,8564,2010-02-22 00:00:00,45,Balanced,NaN,Little,30,Short,Free Form,...,70,Lots,Free Form,30,Deep,35,Press,60,Normal,Offside Trap
1,Milan,8564,2011-02-22 00:00:00,65,Balanced,NaN,Little,50,Mixed,Organised,...,50,Normal,Free Form,50,Medium,50,Press,50,Normal,Offside Trap
2,Milan,8564,2012-02-22 00:00:00,45,Balanced,NaN,Little,50,Mixed,Organised,...,50,Normal,Free Form,45,Medium,45,Press,50,Normal,Cover
3,Milan,8564,2013-09-20 00:00:00,48,Balanced,NaN,Little,54,Mixed,Organised,...,64,Normal,Free Form,48,Medium,49,Press,53,Normal,Cover
4,Milan,8564,2014-09-19 00:00:00,48,Balanced,70.0,Lots,52,Mixed,Organised,...,75,Lots,Organised,58,Medium,57,Press,49,Normal,Cover


In [78]:
team_attributes.shape

(733, 24)

- Null values

In [79]:
team_attributes.isnull().sum()

team_long_name                      0
team_api_id                         0
date                                0
buildUpPlaySpeed                    0
buildUpPlaySpeedClass               0
buildUpPlayDribbling              488
buildUpPlayDribblingClass           0
buildUpPlayPassing                  0
buildUpPlayPassingClass             0
buildUpPlayPositioningClass         0
chanceCreationPassing               0
chanceCreationPassingClass          0
chanceCreationCrossing              0
chanceCreationCrossingClass         0
chanceCreationShooting              0
chanceCreationShootingClass         0
chanceCreationPositioningClass      0
defencePressure                     0
defencePressureClass                0
defenceAggression                   0
defenceAggressionClass              0
defenceTeamWidth                    0
defenceTeamWidthClass               0
defenceDefenderLineClass            0
dtype: int64

In [80]:
team_attributes.shape

(733, 24)

We will remove the column buildUpPlayDribbling because it has a lot of missing values. 488/733 are null value

In [81]:
team_attributes = team_attributes.drop('buildUpPlayDribbling', axis=1)

- Duplicates

In [82]:
team_attributes.duplicated().sum()

0

- Date column

In [83]:
team_attributes['date'] = pd.to_datetime(team_attributes['date'])

In [84]:
team_attributes.head(50)

,team_long_name,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,Milan,8564,2010-02-22,45,Balanced,Little,30,Short,Free Form,55,...,70,Lots,Free Form,30,Deep,35,Press,60,Normal,Offside Trap
1,Milan,8564,2011-02-22,65,Balanced,Little,50,Mixed,Organised,50,...,50,Normal,Free Form,50,Medium,50,Press,50,Normal,Offside Trap
2,Milan,8564,2012-02-22,45,Balanced,Little,50,Mixed,Organised,65,...,50,Normal,Free Form,45,Medium,45,Press,50,Normal,Cover
3,Milan,8564,2013-09-20,48,Balanced,Little,54,Mixed,Organised,51,...,64,Normal,Free Form,48,Medium,49,Press,53,Normal,Cover
4,Milan,8564,2014-09-19,48,Balanced,Lots,52,Mixed,Organised,66,...,75,Lots,Organised,58,Medium,57,Press,49,Normal,Cover
5,Milan,8564,2015-09-10,48,Balanced,Lots,52,Mixed,Organised,66,...,69,Lots,Organised,58,Medium,57,Press,49,Normal,Cover
6,UD Almería,9865,2010-02-22,65,Balanced,Little,45,Mixed,Organised,45,...,70,Lots,Organised,70,High,65,Press,30,Narrow,Cover
7,UD Almería,9865,2011-02-22,55,Balanced,Little,53,Mixed,Organised,57,...,69,Lots,Organised,58,Medium,68,Double,67,Wide,Cover
8,UD Almería,9865,2012-02-22,42,Balanced,Little,70,Long,Organised,57,...,40,Normal,Organised,37,Medium,53,Press,45,Normal,Cover
9,UD Almería,9865,2013-09-20,46,Balanced,Little,65,Mixed,Organised,57,...,40,Normal,Organised,37,Medium,53,Press,45,Normal,Cover


The Team Attributes table may not significantly contribute to our analysis as it is updated only once a year. All the values remain constant throughout the season, but ideally, they should vary based on the team's starting lineup and formation. Therefore, we will exclude this table from our analysis to avoid potential confusion and ensure the accuracy of our model.

## Summary of data cleaning

In the data-cleaning process, the following steps were taken:

1. **Players Data**: The `Player` and `Player_Attributes` tables were merged, keeping only the most important columns. Null values in the 'overall_rating' column were discarded as this column is vital for the analysis. The 'date' and 'birthday' columns were converted to datetime data type and the age of the players at the time of the match was calculated.

2. **Matches Data**: Important data from the `Match,` `League,` and `Team` tables were extracted. Null values in the home and away player columns were analyzed by the league to determine the extent of the missing data. The 'date' column was converted to datetime data type. The player ID was replaced with the overall rating and age. The result of the match was calculated and added as a new column.

3. **Teams Data**: Essential data from the 'Team' and 'Team_Attributes' tables were retrieved. The column 'buildUpPlayDribbling' was removed due to a significant number of missing values. The 'date' column was converted to datetime data type. However, the Team Attributes table was later excluded from the analysis as it was updated only once a year and all the values remained constant throughout the season, a decision made to ensure the accuracy and relevance of the analysis.

# Feature Engineering

- Average rating of team and difference of home and away team rating

In [85]:
for team in ['home', 'away']:
    player_columns = [f'{team}_player_{i}' for i in range(1, 12)]
    matches[f'average_{team}_player_rating'] = matches[player_columns].mean(axis=1).round(2)

In [86]:
matches['players_rating_difference'] = matches['average_home_player_rating'] - matches['average_away_player_rating']

 - Average player age and difference of home and away team age

In [87]:
for team in ['home', 'away']:
    matches[f'{team}_players_age'] = matches[[f'{team}_player_{i}_age' for i in range(1, 12)]].mean(axis=1).round(2)

In [88]:
matches['players_age_difference'] = matches['home_players_age'] - matches['away_players_age']

- Home team and away team in the last five matches of the season form

We will calculate the form of the home and away teams in the last five matches of the season. The form is calculated based on the average number of points earned in the last five matches playing at home or away. The points are calculated as follows: 3 points for a win, 1 point for a draw, and 0 points for a loss. Since the form is calculated based on the last five matches, we will use a rolling window of 5 to calculate the average number of points and shift the result by one to avoid data leakage.

In [89]:
criteria = [
    (matches['result'] == 'Win'),
    (matches['result'] == 'Draw'),
    (matches['result'] == 'Lose')
]

choices_home = [3, 1, 0]
choices_away = [0, 1, 3]

matches['home_team_points'] = np.select(criteria, choices_home)
matches['away_team_points'] = np.select(criteria, choices_away)

In [90]:
for team in ['home', 'away']:
    matches = matches.sort_values(['season', f'{team}_team', 'date'])
    matches[f'{team}_team_form'] = matches.groupby(['season', f'{team}_team'])[f'{team}_team_points'].shift(1).rolling(
        window=5, min_periods=1).mean()

- Team formations

We will calculate the formation of the home and away teams in each match. Y-coordinate represents the player's position on the field. It can be used to determine whether a player is a defender, midfielder, or forward.

In [91]:
def get_formation(positions):
    defenders = np.sum((positions > 1) & (positions <= 4), axis=1).tolist()
    midfielders = np.sum((positions >= 5) & (positions <= 9), axis=1).tolist()
    forwards = np.sum((positions >= 10) & (positions <= 11), axis=1).tolist()
    return [f"{d}-{m}-{f}" for d, m, f in zip(defenders, midfielders, forwards)]


home_positions = matches[[f'home_player_Y{i}' for i in range(1, 12)]].values
away_positions = matches[[f'away_player_Y{i}' for i in range(1, 12)]].values

matches['home_team_formations'] = get_formation(home_positions)
matches['away_team_formations'] = get_formation(away_positions)

- Rolling average of goals scored and conceded

We will calculate the rolling average of goals scored and conceded by each team in the last five matches of the season playing at home or away.

In [92]:
def calculate_team_metrics(df, teams_type):
    grouped_matches = df.groupby(['season', f'{teams_type}_team'])

    df[f'{teams_type}_team_rolling_goals'] = grouped_matches[f'{teams_type}_team_goal'].shift(1).rolling(window=5,
                                                                                                         min_periods=1).mean()
    df[f'{teams_type}_team_rolling_conceded'] = grouped_matches[
        f'away_team_goal' if teams_type == 'home' else 'home_team_goal'].shift(1).rolling(window=5,
                                                                                          min_periods=1).mean()

    return df


team_types = ['home', 'away']

for team in team_types:
    matches = matches.sort_values(['season', f'{team}_team', 'date'])
    matches = calculate_team_metrics(matches, team)

- Goal keeper rating, defenders rating, midfielders rating, and forwards rating 

We will calculate rating of the goalkeepers and average and total rating of defenders, midfielders, and forwards for each team in each match.

In [93]:
matches['Home_GK'] = matches['home_player_1']
matches['Away_GK'] = matches['away_player_1']

In [94]:
def calculate_ratings(match_row, squad, start, end):
    player_cols = [f'{squad}_player_{i}' for i in range(2, 12) if
                   match_row[f'{squad}_player_Y{i}'] in range(start, end)]
    return round(match_row[player_cols].mean(), 2), match_row[player_cols].sum()


player_positions_range = {
    'defenders': (2, 5),
    'midfielders': (5, 10),
    'forwards': (10, 12)
}

for position, (position_start, position_end) in player_positions_range.items():
    for team_type in ['home', 'away']:
        matches[f'{position}_{team_type}_rating'], matches[f'{position}_{team_type}_total_rating'] = zip(
            *matches.apply(lambda match_row: calculate_ratings(match_row, team_type, position_start, position_end),
                           axis=1))

In [95]:
matches.head()

,match_api_id,season,date,home_team_goal,away_team_goal,stage,league_name,home_team,away_team,home_player_1,...,defenders_away_rating,defenders_away_total_rating,midfielders_home_rating,midfielders_home_total_rating,midfielders_away_rating,midfielders_away_total_rating,forwards_home_rating,forwards_home_total_rating,forwards_away_rating,forwards_away_total_rating
7811,499320,2008/2009,2008-08-16,2,1,1,Germany 1. Bundesliga,VfL Wolfsburg,1. FC Köln,70.0,...,71.50,286.0,69.75,279.0,71.0,284.0,75.0,150.0,74.0,148.0
8014,499343,2008/2009,2008-08-29,0,2,3,Germany 1. Bundesliga,Karlsruher SC,1. FC Köln,68.0,...,68.50,274.0,69.00,276.0,69.2,346.0,69.0,69.0,78.0,78.0
8074,499358,2008/2009,2008-09-20,2,0,5,Germany 1. Bundesliga,DSC Arminia Bielefeld,1. FC Köln,62.0,...,70.75,283.0,68.00,272.0,70.2,351.0,73.0,146.0,78.0,78.0
8094,499378,2008/2009,2008-10-04,1,2,7,Germany 1. Bundesliga,Borussia Mönchengladbach,1. FC Köln,71.0,...,70.25,281.0,71.00,355.0,65.0,325.0,70.0,70.0,78.0,78.0
8106,499390,2008/2009,2008-10-24,2,0,9,Germany 1. Bundesliga,Bayer 04 Leverkusen,1. FC Köln,78.0,...,70.25,281.0,71.75,287.0,65.0,325.0,73.5,147.0,78.0,78.0


## Summary of feature engineering

After cleaning the data, several new features were generated. These include the average rating of team players, the difference in home and away team ratings, the average player age, and the difference in home and away team ages. The form of the home and away teams in the last five matches of the season, the team formations, and the rolling average of goals scored and conceded were also calculated. Average and total ratings by position (goalkeepers, defenders, midfielders, and forwards) were computed for each team in each match. These features are expected to enhance the predictive power of our models and provide valuable insights into the performance of the teams and players.

#  Save the dataset

- We will save the dataset to a csv file for further analysis.

In [96]:
matches.to_csv('matches.csv', index=False)